In [4]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [5]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [6]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [7]:
%%capture pwd
!pwd

In [8]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-has-brexit-affected-business-investment-in-the-uk', 300, 500)

# Fig 1

In [15]:
df = pd.read_excel("raw/Data_for_charts.xlsx", sheet_name='Fig1',usecols='A:D')
df.columns=['yearq','bi','ci','it']

In [16]:
f = "fig1_investment_trend"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,yearq,bi,ci,it
0,1997 Q1,34.459,NaN,38.337308
1,1997 Q2,36.590,NaN,38.528995
2,1997 Q3,38.553,NaN,38.721640
3,1997 Q4,39.381,NaN,38.915248
4,1998 Q1,40.131,NaN,39.109824


In [55]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
).transform_calculate(date = "timeParse(replace(replace(replace(replace(datum.yearq, 'Q1', '02'), 'Q2', '05'),  'Q3', '08'), 'Q4', '11'), '%Y %m')")
bi = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "bi:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="£ billion, adjusted for inflation, 2019 constant prices",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[25, 65]),
    )
)
ci=base.mark_line(color=colors["eco-turquiose"]).encode(y="ci:Q")
it=base.mark_line(color=colors["eco-turquiose"],strokeDash=[5,5]).encode(y="it:Q")
hline=base.mark_rule(color=colors["eco-gray"],opacity=0.5).transform_filter('datum.yearq=="2016 Q1"')
label1=bi.mark_text(text='Business investment',color=colors["eco-mid-blue"],yOffset=30).transform_filter('datum.yearq=="2010 Q1"')
label2=ci.mark_text(text='Counterfactual investment',color=colors["eco-turquiose"],yOffset=-20).transform_filter('datum.yearq=="2019 Q1"')
label3=it.mark_text(text='Memo: Investment trend',color=colors["eco-turquiose"],yOffset=-45).transform_filter('datum.yearq=="2009 Q1"')

layer1 = (
    ((bi+ci+it+hline+label1+label2+label3).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

# Fig 2

In [60]:
df = pd.read_excel("raw/Data_for_charts.xlsx", sheet_name='Fig2',usecols='A:C')
df.columns=['yearq','uk','dg']

In [61]:
f = "fig2_brexit"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,yearq,uk,dg
0,2009 Q1,76.320,74.362845
1,2009 Q2,73.206,72.908874
2,2009 Q3,74.047,72.650739
3,2009 Q4,72.917,72.351221
4,2010 Q1,75.936,71.006322


In [122]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
).transform_calculate(date = "timeParse(replace(replace(replace(replace(datum.yearq, 'Q1', '02'), 'Q2', '05'),  'Q3', '08'), 'Q4', '11'), '%Y %m')")
uk = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "uk:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="£ billion, adjusted for inflation, chained volume measures",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[60, 120]),
    )
)
dg=base.mark_line(color=colors["eco-turquiose"]).encode(y="dg:Q")
hline=base.mark_rule(color=colors["eco-gray"],opacity=0.5).transform_filter('datum.yearq=="2016 Q3"')
hline2=base.mark_rule(color=colors["eco-gray"],opacity=0.5).transform_filter('datum.yearq=="2021 Q1"')
label1=uk.mark_text(text='UK',color=colors["eco-mid-blue"],yOffset=60,xOffset=0,align='right')\
    .transform_filter('datum.yearq=="2020 Q1"')
label2=dg.mark_text(text='Doppelgänger UK',color=colors["eco-turquiose"],yOffset=90,xOffset=-5,align='right')\
    .transform_filter('datum.yearq=="2020 Q1"')
label3=hline.mark_text(text='EU Referendum',color=colors["eco-gray"],xOffset=-5,yOffset=135,align='right')
label4=hline2.mark_text(text='Single market exit',color=colors["eco-gray"],xOffset=-5,yOffset=135,align='right')
sf=base.mark_rule(color=colors["eco-dot"]).transform_filter('datum.yearq=="2022 Q2"').encode(
    y='uk:Q',y2='dg:Q'
)
label5=sf.mark_text(text='UK shortfall',color=colors["eco-dot"],xOffset=5,yOffset=-55,align='left')
label6=sf.mark_text(text='in 2022 Q2',color=colors["eco-dot"],xOffset=5,yOffset=-40,align='left')
label7=sf.mark_text(text='11% or',color=colors["eco-dot"],xOffset=5,yOffset=-20,align='left')
label8=sf.mark_text(text='£12 billion',color=colors["eco-dot"],xOffset=5,yOffset=-5,align='left')

layer1 = (
    ((uk+dg+hline+hline2+label1+label2+label3+label4+sf+label5+label6+label7+label8
     ).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

WARN y2 dropped as it is incompatible with "text".
WARN y2 dropped as it is incompatible with "text".
WARN y2 dropped as it is incompatible with "text".
WARN y2 dropped as it is incompatible with "text".
/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
WARN y2 dropped as it is incompatible with "text".
WARN y2 dropped as it is incompatible with "text".
WARN y2 dropped as it is incompatible with "text".
WARN y2 dropped as it is incompatible with "text".
/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

# Fig 3

In [130]:
df = pd.read_excel("raw/Data_for_charts.xlsx", sheet_name='Fig3',usecols='A:H')
df=df.rename(columns={'Unnamed: 0':'yearq'})
df=df.set_index('yearq').stack().reset_index()
df.columns=['yearq','country','value']

In [131]:
f = "fig3_countries"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,yearq,country,value
0,2016 Q1,UK,100.350286
1,2016 Q1,Germany,99.924184
2,2016 Q1,France,100.474252
3,2016 Q1,Italy,99.640369
4,2016 Q1,US,98.954291


In [161]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
).transform_calculate(date = "timeParse(replace(replace(replace(replace(datum.yearq, 'Q1', '02'), 'Q2', '05'),  'Q3', '08'), 'Q4', '11'), '%Y %m')")
line = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Index, 2016 = 100",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[75, 135]),
    ),
    color=alt.Color('country:N',scale=alt.Scale(
        range=['#EF3340','#0055A4','#FFCD00','#007A33','gray','#012169','#BF0D3E']
        ),legend=None
    )
)
labels=line.mark_text(align='left',xOffset=5).encode(text='country:N')
labels1=labels.transform_filter('datum.yearq=="2022 Q4"')
labels2=labels.transform_filter('datum.yearq=="2022 Q3"').transform_filter('datum.country=="Germany"')
labels3=labels.transform_filter('datum.yearq=="2022 Q3"').transform_filter('datum.country=="Italy"')
labels4=labels.mark_text(align='left',yOffset=15,xOffset=-15).transform_filter('datum.yearq=="2022 Q3"').transform_filter('datum.country=="Canada"')

layer1 = (
    ((line+labels1+labels2+labels3+labels4
     ).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)